In [ ]:
import pandas as pd
import random

historia:
loty startuja z ziemi - poczatek ukladu wspolrzednych.
im planeta w galaktyce dalej od ziemii tym:
- wiecej pracownikow
- drozszy lot
- mozliwe ze bedzie dostepny tylko w jedna strone

Im:
- statek ma wieksza capacity (wielkosc),
-
tym cena wyzsza

planety w jednej galaktyce maja zblizone wspolrzedne.


jak ktos pojechal tylko w jedna strone - to juz nie moze pojawic sie w transakcjach i bookingach.

niech salary pracownikow bedzie najwyzsze dla commander a najnizsze dla technitian i neich zalezy od doswiadczenia w firmie.



planets + coordinates + galaxy

In [14]:
# Regenerate planets with galaxy-based coordinate clustering

# Step 1: Assign base coordinates per galaxy
galaxies = [
    "Andros Spiral", "Helion Prime", "Novara Cluster", "Vortexis", "Celestara",
    "Omega Verge", "Halcyon Drift", "Nemera Arc", "Quasari Belt", "Eclipse Halo",
    "Astralis", "Kronis Ring", "Auroria", "Zerion Reach", "Polaris Nexus"
]

# Assign galaxy_id
galaxy_data = [{"galaxy_id": i + 1, "galaxy_name": name} for i, name in enumerate(galaxies)]
planet_names = [
    "Aetheria", "Zorath", "Nyxos", "Virella", "Targus",
    "Ecliptor", "Orionis", "Caelora", "Draconis", "Pyrrhos",
    "Talmera", "Lunaris", "Zephyra", "Solvana", "Obscura",
    "Cryon", "Valtor", "Serenos", "Arctyra", "Phobaris"
]
galaxy_coords = {}
for galaxy in galaxy_data:
    galaxy_coords[galaxy["galaxy_id"]] = {
        "base_x": random.uniform(-1000, 1000),
        "base_y": random.uniform(-1000, 1000),
        "base_z": random.uniform(-1000, 1000)
    }

# Step 2: Generate planets with coordinates near their galaxy center
clustered_planet_data = []
for i, name in enumerate(planet_names):
    galaxy_id = random.randint(1, len(galaxies))
    base = galaxy_coords[galaxy_id]
    clustered_planet_data.append({
        "planet_id": i + 1,
        "planet_name": name,
        "coordinates": f"X:{base['base_x'] + random.uniform(-100, 100):.2f}, Y:{base['base_y'] + random.uniform(-100, 100):.2f}, Z:{base['base_z'] + random.uniform(-100, 100):.2f}",
        "galaxy_id": galaxy_id
    })

# Create and save new DataFrame
df_clustered_planets = pd.DataFrame(clustered_planet_data)


In [17]:
# Re-run after code execution environment reset
import pandas as pd
import random

# Galaxies and their base coordinates (for clustering)
galaxies = [
    "Andros Spiral", "Helion Prime", "Novara Cluster", "Vortexis", "Celestara",
    "Omega Verge", "Halcyon Drift", "Nemera Arc", "Quasari Belt", "Eclipse Halo",
    "Astralis", "Kronis Ring", "Auroria", "Zerion Reach", "Polaris Nexus"
]

# Step 1: Generate galaxy table
galaxy_data = []
galaxy_coords = {}

for i, name in enumerate(galaxies, start=1):
    galaxy_data.append({
        "galaxy_id": i,
        "galaxy": name
    })
    # base coordinates for clustered planets
    galaxy_coords[i] = {
        "base_x": random.uniform(-1000, 1000),
        "base_y": random.uniform(-1000, 1000),
        "base_z": random.uniform(-1000, 1000)
    }

# Step 2: Generate planets clustered by galaxy
planet_names = [
    "Aetheria", "Zorath", "Nyxos", "Virella", "Targus", "Ecliptor", "Orionis", "Caelora",
    "Draconis", "Pyrrhos", "Talmera", "Lunaris", "Zephyra", "Solvana", "Obscura",
    "Cryon", "Valtor", "Serenos", "Arctyra", "Phobaris"
]

planet_data = []
for i, name in enumerate(planet_names):
    galaxy_id = random.randint(1, len(galaxies))
    base = galaxy_coords[galaxy_id]
    planet_data.append({
        "planet_id": i + 1,
        "planet": name,
        "x_coord": round(base["base_x"] + random.uniform(-100, 100), 2),
        "y_coord": round(base["base_y"] + random.uniform(-100, 100), 2),
        "z_coord": round(base["base_z"] + random.uniform(-100, 100), 2),
        "galaxy_id": galaxy_id
    })

# Convert to DataFrames
df_galaxies = pd.DataFrame(galaxy_data)
df_planets = pd.DataFrame(planet_data)

# Save to CSV
# df_galaxies.to_csv("/mnt/data/galaxies_from_diagram.csv", index=False)
# df_planets.to_csv("/mnt/data/planets_from_diagram.csv", index=False)

# # Show preview
# import ace_tools as tools; tools.display_dataframe_to_user(name="Planets from ERD", dataframe=df_planets)


In [19]:
df_planets

,planet_id,planet,x_coord,y_coord,z_coord,galaxy_id
0,1,Aetheria,-278.92,770.26,-34.32,13
1,2,Zorath,822.24,100.69,902.98,5
2,3,Nyxos,934.10,100.37,808.99,5
3,4,Virella,-884.22,32.77,321.90,10
4,5,Targus,-313.02,825.37,-543.77,6
5,6,Ecliptor,-293.45,902.17,-527.32,6
6,7,Orionis,513.09,682.07,48.99,9
7,8,Caelora,-178.60,-8.56,-184.89,12
8,9,Draconis,423.57,-398.67,427.10,8
9,10,Pyrrhos,-528.67,-281.33,160.45,2


trip types

In [16]:
import math

# Step 1: Generate a trip type for each planet
trip_types = []

for i, row in df_planets.iterrows():
    distance_from_origin = math.sqrt(row["x_coord"]**2 + row["y_coord"]**2 + row["z_coord"]**2)
    is_round_trip = distance_from_origin < 850  # one-way if far from (0,0,0)
    
    trip_types.append({
        "triptype_id": i + 1,
        "name": f"Mission to {row['planet']}",
        "coordinates": f"X:{row['x_coord']}, Y:{row['y_coord']}, Z:{row['z_coord']}",
        "description": f"Explore the unique surface and atmosphere of planet {row['planet']}.",
        "is_round_trip": is_round_trip,
        "planet_id": row["planet_id"]
    })

# Convert to DataFrame and save
df_trip_types = pd.DataFrame(trip_types)
#df_trip_types.to_csv("/mnt/data/trip_types_linked_to_planets.csv", index=False)

# Show preview
#import ace_tools as tools; tools.display_dataframe_to_user(name="Trip Types Linked to Planets", dataframe=df_trip_types)


NameError: name 'df_planets' is not defined

trips + emplyee +rocket

In [ ]:
# Update employee and trip generation to ensure each trip has a Pilot and Navigator

trips = []
trip_employees = []

for i, row in df_trip_types.iterrows():
    num_variants = random.randint(1, 3)  # 1-3 trips per trip type
    for _ in range(num_variants):
        trip_id = len(trips) + 1
        departure = start_date + timedelta(days=random.randint(0, 365))
        duration = random.randint(5, 30)
        return_date = departure + timedelta(days=duration)
        rocket_id = random.randint(1, len(rockets))

        trips.append({
            "trip_id": trip_id,
            "triptype_id": row["triptype_id"],
            "rocket_id": rocket_id,
            "departure_date": departure.strftime("%Y-%m-%d"),
            "return_date": return_date.strftime("%Y-%m-%d"),
            "status": random.choice(["Scheduled", "Completed", "Delayed"])
        })

        is_round_trip = row["is_round_trip"]
        min_roles = ["Pilot", "Navigator"]
        other_roles = ["Technician", "Mission Specialist", "Commander"]
        n_optional = random.randint(1, 2) if is_round_trip else 0  # more if round trip

        assigned = []
        # Add required roles
        for role in min_roles:
            available = [e for e in employees if e["position"] == role and e["employee_id"] not in assigned]
            if available:
                emp = random.choice(available)
                assigned.append(emp["employee_id"])
                trip_employees.append({
                    "trip_id": trip_id,
                    "employee_id": emp["employee_id"],
                    "role_in_trip": emp["position"]
                })

        # Add optional roles
        for _ in range(n_optional):
            available = [e for e in employees if e["position"] in other_roles and e["employee_id"] not in assigned]
            if available:
                emp = random.choice(available)
                assigned.append(emp["employee_id"])
                trip_employees.append({
                    "trip_id": trip_id,
                    "employee_id": emp["employee_id"],
                    "role_in_trip": emp["position"]
                })

# Update DataFrames and save
df_trips = pd.DataFrame(trips)
df_trip_employees = pd.DataFrame(trip_employees)
df_employees = pd.DataFrame(employees)
df_rockets = pd.DataFrame(rockets)

# df_trips.to_csv("/mnt/data/trips_generated.csv", index=False)
# df_trip_employees.to_csv("/mnt/data/trip_employees.csv", index=False)
# df_employees.to_csv("/mnt/data/employees_generated.csv", index=False)
# df_rockets.to_csv("/mnt/data/rockets_generated.csv", index=False)

# # Preview assigned trip employees
# import ace_tools as tools; tools.display_dataframe_to_user(name="Trip Employees with Required Roles", dataframe=df_trip_employees)


transaction

In [ ]:
import math

# Step 1: Generate clients
from faker import Faker
fake = Faker()
clients = [{
    "client_id": i + 1,
    "first_name": fake.first_name(),
    "last_name": fake.last_name(),
    "email": fake.email(),
    "phone": fake.phone_number()
} for i in range(50)]

df_clients = pd.DataFrame(clients)

# Step 2: Generate bookings (1-3 per client, random trips)
bookings = []
for client in clients:
    n_bookings = random.randint(1, 3)
    booked_trip_ids = random.sample(df_trips["trip_id"].tolist(), n_bookings)
    for trip_id in booked_trip_ids:
        bookings.append({
            "booking_id": len(bookings) + 1,
            "client_id": client["client_id"],
            "trip_id": trip_id,
            "seat_number": random.randint(1, 12)
        })

df_bookings = pd.DataFrame(bookings)

# Step 3: Generate transactions
# Base price logic: more bookings, round trip, distance, capacity increase cost

# Planet coordinates (from triptype -> planet)
planet_coords = df_trip_types.set_index("triptype_id")["coordinates"].to_dict()
round_trip_flags = df_trip_types.set_index("triptype_id")["is_round_trip"].to_dict()
triptypes_planet = df_trip_types.set_index("triptype_id")["planet_id"].to_dict()
rockets_capacity = df_rockets.set_index("rocket_id")["capacity"].to_dict()

# Count number of bookings per trip
booking_counts = df_bookings.groupby("trip_id").size().to_dict()

transactions = []
for booking in bookings:
    trip_id = booking["trip_id"]
    trip_row = df_trips[df_trips["trip_id"] == trip_id].iloc[0]
    triptype_id = trip_row["triptype_id"]
    rocket_id = trip_row["rocket_id"]
    coords = planet_coords[triptype_id]

    # Calculate distance
    coord_parts = [float(val.split(":")[1]) for val in coords.split(", ")]
    distance = math.sqrt(sum(c ** 2 for c in coord_parts))

    base_price = 10000
    base_price += booking_counts[trip_id] * 200     # more passengers → higher cost
    base_price += (distance / 10)                   # longer trip → more expensive
    base_price += rockets_capacity[rocket_id] * 100 # bigger rocket → more cost
    if round_trip_flags[triptype_id]:
        base_price *= 1.25  # round trip multiplier

    transactions.append({
        "transaction_id": len(transactions) + 1,
        "client_id": booking["client_id"],
        "trip_id": trip_id,
        "amount": round(base_price, 2),
        "payment_date": fake.date_between(start_date='-1y', end_date='today').strftime("%Y-%m-%d"),
        "method": random.choice(["card", "transfer", "crypto"])
    })

df_transactions = pd.DataFrame(transactions)

# Save all new CSVs
# df_clients.to_csv("/mnt/data/clients_generated.csv", index=False)
# df_bookings.to_csv("/mnt/data/bookings_generated.csv", index=False)
# df_transactions.to_csv("/mnt/data/transactions_generated.csv", index=False)

# # Preview transactions
# import ace_tools as tools; tools.display_dataframe_to_user(name="Generated Transactions", dataframe=df_transactions)


cost

In [ ]:

# Count employees per trip
employee_counts = df_trip_employees.groupby("trip_id").size().to_dict()
# Sum of transactions per trip
income_per_trip = df_transactions.groupby("trip_id")["amount"].sum().to_dict()
# Triptype info
planet_coords = df_trip_types.set_index("triptype_id")["coordinates"].to_dict()
round_trip_flags = df_trip_types.set_index("triptype_id")["is_round_trip"].to_dict()

costs = []
for trip in df_trips.itertuples():
    trip_id = trip.trip_id
    triptype_id = trip.triptype_id
    coords = planet_coords[triptype_id]
    coord_parts = [float(val.split(":")[1]) for val in coords.split(", ")]
    distance = math.sqrt(sum(c ** 2 for c in coord_parts))
    is_round = round_trip_flags[triptype_id]
    n_employees = employee_counts.get(trip_id, 1)
    max_total_cost = income_per_trip.get(trip_id, 10000)

    # Base cost estimate
    base_cost = 5000 + (n_employees * 300) + (distance / 20)
    if is_round:
        base_cost *= 1.2

    # Generate 2-4 individual costs that sum up to less than income
    total_cost = min(base_cost, max_total_cost * 0.9)
    n_costs = random.randint(2, 4)
    cost_shares = [random.uniform(0.15, 0.35) for _ in range(n_costs)]
    total_share = sum(cost_shares)
    normalized_shares = [s / total_share for s in cost_shares]

    for i in range(n_costs):
        costs.append({
            "cost_id": len(costs) + 1,
            "trip_id": trip_id,
            "cost_type": random.choice(["Fuel", "Crew", "Maintenance", "Logistics"]),
            "amount": round(total_cost * normalized_shares[i], 2)
        })

df_costs = pd.DataFrame(costs)
# df_costs.to_csv("/mnt/data/costs_generated.csv", index=False)

# # Preview costs
# import ace_tools as tools; tools.display_dataframe_to_user(name="Generated Costs", dataframe=df_costs)


clients + adress + emergency_contact

In [ ]:
# Filter clients based on their bookings and trip directions
# Also identify the two galaxies with most round-trip clients

# First, merge trip info into bookings
df_booking_trips = df_bookings.merge(df_trips, on="trip_id", how="left")
df_booking_trips = df_booking_trips.merge(df_trip_types[["triptype_id", "is_round_trip", "planet_id"]], on="triptype_id", how="left")
df_booking_trips = df_booking_trips.merge(df_planets[["planet_id", "galaxy_id"]], on="planet_id", how="left")

# Step 1: Remove clients who took at least one one-way trip
one_way_clients = df_booking_trips[df_booking_trips["is_round_trip"] == False]["client_id"].unique()
df_clients_filtered = df_clients[~df_clients["client_id"].isin(one_way_clients)]

# Step 2: For remaining clients, count round-trip bookings per galaxy
round_trip_bookings = df_booking_trips[
    (df_booking_trips["is_round_trip"] == True) &
    (df_booking_trips["client_id"].isin(df_clients_filtered["client_id"]))
]

galaxy_counts = round_trip_bookings.groupby("galaxy_id").size().sort_values(ascending=False)
top_two_galaxies = galaxy_counts.head(2).index.tolist()

# Step 3: Select clients who traveled to those galaxies
top_clients = round_trip_bookings[round_trip_bookings["galaxy_id"].isin(top_two_galaxies)]["client_id"].unique()
df_clients_top = df_clients_filtered[df_clients_filtered["client_id"].isin(top_clients)]

# Step 4: Add fake address and emergency contact info
from faker import Faker
fake = Faker()

addresses = []
emergency_contacts = []

for client_id in df_clients_top["client_id"]:
    addresses.append({
        "client_id": client_id,
        "street": fake.street_address(),
        "city": fake.city(),
        "postal_code": fake.postcode(),
        "country": fake.country()
    })

    emergency_contacts.append({
        "client_id": client_id,
        "name": fake.name(),
        "phone": fake.phone_number(),
        "email": fake.email()
    })

df_addresses = pd.DataFrame(addresses)
df_emergency_contacts = pd.DataFrame(emergency_contacts)

# Save results
df_clients_top.to_csv("/mnt/data/final_clients.csv", index=False)
df_addresses.to_csv("/mnt/data/final_addresses.csv", index=False)
df_emergency_contacts.to_csv("/mnt/data/final_emergency_contacts.csv", index=False)

# Show clients selected
import ace_tools as tools; tools.display_dataframe_to_user(name="Final Clients (Round Trip & Top Galaxies)", dataframe=df_clients_top)
